In [1]:
import pandas as pd
import io
import os
import pandas as pd
import librosa
from tqdm.notebook import tqdm

In [5]:
#### Loading the Audio Files ###

# Path to the directory containing the audio files
folder_path = "folder-path"

# List all files in the directory
file_names = os.listdir(folder_path)

# Filter out only the audio files (assuming they have extensions like .mp3, .wav, etc.)
audio_files = [file for file in file_names if file.endswith('.mp3') or file.endswith('.wav')]

# Initialize an empty list to store audio file metadata
audio_data_list = []

# Iterate over each audio file and load it
for audio_file in audio_files:
    # Construct the full path to the audio file
    audio_file_path = os.path.join(folder_path, audio_file)
    
    # Load the audio file
    audio_data, sampling_rate = librosa.load(audio_file_path)
    
    # Normalize audio data to the range [-1, 1]
    audio_data_normalized = librosa.util.normalize(audio_data)
  
    # Extract metadata such as duration, sampling rate, etc.
    duration = librosa.get_duration(y=audio_data_normalized, sr=sampling_rate)
    
    # Append metadata to the list
    audio_data_list.append({
        'audio': audio_file,
#         'file_path': audio_file_path,
#         'duration': duration,
        'sampling_rate': sampling_rate
    })

# Create a DataFrame from the list of audio metadata
df1 = pd.DataFrame(audio_data_list)
df1

In [2]:
#### Loading the reference text for the Audio file ###

ref = []
with io.open("text",encoding='utf-8') as textFile:
    try:
        for line in textFile:
#             num = line[1:line.find(" ")]
            text = line[line.find(" "):]
            ref.append(text)
#             print(text)
          
    except:
        print("failed")
        
df2 = pd.DataFrame(ref,columns=['Reference'])
df2

In [62]:
### Combining The Both Dataset Together ###
audio_df = df2.combine_first(df1)
audio_df

,Reference,audio,sampling_rate
0,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा ...,01-00004-02.mp3,22050
1,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या ...,01-00008-03.mp3,22050
2,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल म...,01-00031-03.mp3,22050
3,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ...,01-00071-02.mp3,22050
4,समस्या को सुना जायेगा और महिलाओं की समस्या मह...,01-00078-01.mp3,22050
...,...,...,...
1027,नमस्कार श्रोता मैं मोबाइल मिडिया रिपोर्टर खुर...,13-00608-02.mp3,22050
1028,की सरकारी कार्यालयों में बढ़ती भ्रष्टाचार का म...,13-00631-03.mp3,22050
1029,नमस्कार श्रोताओ में मैं धनबाद मोबाइल मीडिया स...,13-00649-03.mp3,22050
1030,इसी के साथ बताना चाहता हूँ कि कोई ध्यान नहीं ...,13-00655-01.mp3,22050


In [11]:
### Selecting The Hindi value for the Whisper Model###


import ipywidgets as widgets
languages = {"hi_in": "Hindi"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="hi_in",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=2, options=(('Select language', None), ('----------', None), ('Hindi (…

In [12]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Hindi (hi_in)


In [13]:
### Loading The Whisper Medium Model ###
import whisper
model = whisper.load_model("medium")

In [14]:
### Creating An method to transcribe ####
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)

In [15]:
dataset = audio_df["audio"]
ref_dataset = audio_df["Reference"]

In [72]:
### intialze the small value for the checking the working of the model ###
process_data = dataset[:164]
ref_data = df2[:164]

In [ ]:
### Processing the Audio Files ###
transcriptions = [] # intailize a empty list to store the transibe text

for audio in tqdm(process_data):
    transcription = model.transcribe(audio, **transcribe_options)
    transcriptions.append(transcription)

In [74]:
### Viewing the DataFrame after trannscriptions ####
data = pd.DataFrame(transcriptions)
data

,text,segments,language
0,वानी रहूल कुमार पुकेड् कुमार समीव्र कुमार रहि...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 15....",Hindi
1,विवाक्राम जिमजारी हैं इसके लिए बहुत समझ या बह...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.2...",Hindi
2,क्योंसे अमाई प्लिक्चा फेजबॉक और गुशे सॉसल मिड...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.7...",Hindi
3,नाले जाम हैं। जिससे निकल्ला दंदा पाने सर्पो प...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 8.5...",Hindi
4,समस्तिया के सुना जाएगा और महलां की समस्तिया म...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 6.0...",Hindi
...,...,...,...
160,जार जन्वरी को परस्वनि वा दह्यारी पंचाइत् बार ...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.2...",Hindi
161,ताल 2013 में 25 दिवीनी पहुता था।,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.3...",Hindi
162,इस घड़ना को से लिप्टरा कर लिए गया है और काप उ...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 7.6...",Hindi
163,भिजाल जारी अजिष्टम् ताप्मान का ट्रेंट जिर्मू ...,"[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.4...",Hindi


In [ ]:
### Dropping the umwanted Columns ###
columns_to_drop = [,'segments','language']
data.drop(columns=columns_to_drop, inplace=True)

In [36]:
data

,text
0,वानी रहूल कुमार पुकेड् कुमार समीव्र कुमार रहि...
1,विवाक्राम जिमजारी हैं इसके लिए बहुत समझ या बह...
2,क्योंसे अमाई प्लिक्चा फेजबॉक और गुशे सॉसल मिड...
3,नाले जाम हैं। जिससे निकल्ला दंदा पाने सर्पो प...
4,समस्तिया के सुना जाएगा और महलां की समस्तिया म...
...,...
160,जार जन्वरी को परस्वनि वा दह्यारी पंचाइत् बार ...
161,ताल 2013 में 25 दिवीनी पहुता था।
162,इस घड़ना को से लिप्टरा कर लिए गया है और काप उ...
163,भिजाल जारी अजिष्टम् ताप्मान का ट्रेंट जिर्मू ...


In [60]:
### Renaming the Columns ####
new_column_names = {
    'text': 'hypothesis'
}
data.rename(columns=new_column_names, inplace=True)
data

,hypothesis
0,वानी रहूल कुमार पुकेड् कुमार समीव्र कुमार रहि...
1,विवाक्राम जिमजारी हैं इसके लिए बहुत समझ या बह...
2,क्योंसे अमाई प्लिक्चा फेजबॉक और गुशे सॉसल मिड...
3,नाले जाम हैं। जिससे निकल्ला दंदा पाने सर्पो प...
4,समस्तिया के सुना जाएगा और महलां की समस्तिया म...
...,...
160,जार जन्वरी को परस्वनि वा दह्यारी पंचाइत् बार ...
161,ताल 2013 में 25 दिवीनी पहुता था।
162,इस घड़ना को से लिप्टरा कर लिए गया है और काप उ...
163,भिजाल जारी अजिष्टम् ताप्मान का ट्रेंट जिर्मू ...


In [46]:
### Combining the two dataset ####
combined_df = pd.concat([data, ref_data], axis=1)
combined_df

,hypothesis,Reference
0,वानी रहूल कुमार पुकेड् कुमार समीव्र कुमार रहि...,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा ...
1,विवाक्राम जिमजारी हैं इसके लिए बहुत समझ या बह...,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या ...
2,क्योंसे अमाई प्लिक्चा फेजबॉक और गुशे सॉसल मिड...,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल म...
3,नाले जाम हैं। जिससे निकल्ला दंदा पाने सर्पो प...,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ...
4,समस्तिया के सुना जाएगा और महलां की समस्तिया म...,समस्या को सुना जायेगा और महिलाओं की समस्या मह...
...,...,...
160,जार जन्वरी को परस्वनि वा दह्यारी पंचाइत् बार ...,ग्यारह जनवरी को परसवनी व दहियारी पंचायत बारह ...
161,ताल 2013 में 25 दिवीनी पहुता था।,साल दो हजार तेरह में पच्चीस डिगरी पहुँचा था\n
162,इस घड़ना को से लिप्टरा कर लिए गया है और काप उ...,इस घटना को से निपटारा कर लिया है और काबू पा ल...
163,भिजाल जारी अजिष्टम् ताप्मान का ट्रेंट जिर्मू ...,फ़िलहाल जारी अधिकतम तापमान का ट्रेंड गेहूं और ...


In [56]:
### Converting the floating type to the string type ####
combined_df['Reference'] = combined_df['Reference'].astype(str)
combined_df['hypothesis'] = combined_df['hypothesis'].astype(str)


wer_scores = [] # intialze a empty list to store the wer score

### Calculating the WER score for each entry ###
for index, row in combined_df.iterrows():
    wer = jiwer.wer(row['Reference'], row['hypothesis'])
    wer_scores.append(wer)

In [57]:
### Adding a new column to the dataframe which cointain the wer score ###
combined_df['wer'] = wer_scores

In [68]:
combined_df

,hypothesis,Reference,wer
0,वानी रहूल कुमार पुकेड् कुमार समीव्र कुमार रहि...,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा ...,0.857143
1,विवाक्राम जिमजारी हैं इसके लिए बहुत समझ या बह...,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या ...,1.117647
2,क्योंसे अमाई प्लिक्चा फेजबॉक और गुशे सॉसल मिड...,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल म...,0.666667
3,नाले जाम हैं। जिससे निकल्ला दंदा पाने सर्पो प...,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर ...,0.708333
4,समस्तिया के सुना जाएगा और महलां की समस्तिया म...,समस्या को सुना जायेगा और महिलाओं की समस्या मह...,0.521739
...,...,...,...
159,मैं राजीतुमा शार्कुर लिला मुंवेर मुंवेर की अव...,मैं राजीव कुमार ठाकुर जिला मुंगेर मुंगेर की आ...,0.714286
160,जार जन्वरी को परस्वनि वा दह्यारी पंचाइत् बार ...,ग्यारह जनवरी को परसवनी व दहियारी पंचायत बारह ...,0.809524
161,ताल 2013 में 25 दिवीनी पहुता था।,साल दो हजार तेरह में पच्चीस डिगरी पहुँचा था\n,0.888889
162,इस घड़ना को से लिप्टरा कर लिए गया है और काप उ...,इस घटना को से निपटारा कर लिया है और काबू पा ल...,0.680000
